In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder
import pandas as pd

# Criar um SparkSession
spark = SparkSession.builder.appName("titanic").getOrCreate()

25/02/15 18:59:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/titanic.csv")

In [5]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [6]:
%%sql

CREATE DATABASE IF NOT EXISTS ml_titanic;

25/02/15 18:50:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [26]:
# Determinar o ponto de divisão (exemplo: 50%)
split_index = len(df) // 2

# Dividir o DataFrame
df1 = df.iloc[:split_index]  # Primeira metade
df2 = df.iloc[split_index:]  # Segunda metade

In [27]:
df1.shape, df2.shape

((445, 12), (446, 12))

In [50]:
spark_df = spark.createDataFrame(df1)
print(spark_df.count())
cols_to_drop = ["PassengerId", "Name", "Ticket", "Cabin"]
spark_df = spark_df.drop(*cols_to_drop)

# 2. Remover linhas com valores nulos
spark_df = spark_df.dropna()

# 3. Transformar variáveis categóricas em numéricas
indexer_sex = StringIndexer(inputCol="Sex", outputCol="Sex_Indexed", handleInvalid="keep")
indexer_embarked = StringIndexer(inputCol="Embarked", outputCol="Embarked_Indexed", handleInvalid="keep")

spark_df = indexer_sex.fit(spark_df).transform(spark_df)
spark_df = indexer_embarked.fit(spark_df).transform(spark_df)

# Remover colunas categóricas originais
spark_df_silver = spark_df.drop("Sex", "Embarked")

# Mostrar resultado
#spark_df_silver.show(5)
spark_df_silver.write.mode("overwrite").saveAsTable("ml_titanic.refined_titanic")

445


In [51]:
%%sql
select count(1) from ml_titanic.refined_titanic

count(1)
357


In [52]:
spark_df = spark.createDataFrame(df2)
print(spark_df.count())
cols_to_drop = ["PassengerId", "Name", "Ticket", "Cabin"]
spark_df = spark_df.drop(*cols_to_drop)

# 2. Remover linhas com valores nulos
spark_df = spark_df.dropna()

# 3. Transformar variáveis categóricas em numéricas
indexer_sex = StringIndexer(inputCol="Sex", outputCol="Sex_Indexed", handleInvalid="keep")
indexer_embarked = StringIndexer(inputCol="Embarked", outputCol="Embarked_Indexed", handleInvalid="keep")

spark_df = indexer_sex.fit(spark_df).transform(spark_df)
spark_df = indexer_embarked.fit(spark_df).transform(spark_df)

# Remover colunas categóricas originais
spark_df_silver = spark_df.drop("Sex", "Embarked")

# Mostrar resultado
#spark_df_silver.show(5)
#spark_df_silver.write.mode("append").saveAsTable("ml_titanic.refined_titanic")
spark_df_silver.write \
  .format("iceberg") \
  .mode("append") \
  .save("ml_titanic.refined_titanic")


446


In [54]:
%%sql
select count(1) from ml_titanic.refined_titanic

count(1)
714


In [55]:
%%sql
SELECT * FROM ml_titanic.refined_titanic.history;

made_current_at,snapshot_id,parent_id,is_current_ancestor
2025-02-15 19:24:13.323000,374014226229711562,None,True
2025-02-15 19:24:16.521000,6262115738980720977,374014226229711562,True


In [57]:
%%sql
SELECT count(1) 
FROM ml_titanic.refined_titanic FOR SYSTEM_VERSION AS OF 374014226229711562;


count(1)
357


In [58]:
%%sql
SELECT count(1) 
FROM ml_titanic.refined_titanic FOR SYSTEM_VERSION AS OF 6262115738980720977;

count(1)
714
